W projekcie będę pracował na zbiorze: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing.

Plan pracy:
    Proces przetwarzania danych.
    Proces modelowania (znajdowania optymalnych parametrów).
    Porównanie modeli.
    Wizualizacja wyników.
    Podsumowanie pracy


In [1]:
print("hello world")

hello world
